In [ ]:
#UNIVERSITY RANKINGS- Categorical Data and PCA
#The dataset on American college and university rankings (available from www.dataminingbook.com) contains information on 1302 American colleges and universities
#offering an undergraduate program. For each university, there are 17 measurements that include continuous measurements (such as
#tuition and graduation rate) and categorical measurements (such as location by state and whether it is a private or a public school).
#a. Remove all categorical variables. Then remove all records with missing numerical measurements from the dataset.
#b. Conduct a principal components analysis on the cleaned data

In [30]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pylab as plt

In [31]:
df = pd.read_csv('Universities.csv')

In [32]:
df.shape

(1302, 20)

In [33]:
df.index

RangeIndex(start=0, stop=1302, step=1)

In [34]:
df.head()

,College Name,State,Public (1)/ Private (2),# appli. rec'd,# appl. accepted,# new stud. enrolled,% new stud. from top 10%,% new stud. from top 25%,# FT undergrad,# PT undergrad,in-state tuition,out-of-state tuition,room,board,add. fees,estim. book costs,estim. personal $,% fac. w/PHD,stud./fac. ratio,Graduation rate
0,Alaska Pacific University,AK,2,193.0,146.0,55.0,16.0,44.0,249.0,869.0,7560.0,7560.0,1620.0,2500.0,130.0,800.0,1500.0,76.0,11.9,15.0
1,University of Alaska at Fairbanks,AK,1,1852.0,1427.0,928.0,NaN,NaN,3885.0,4519.0,1742.0,5226.0,1800.0,1790.0,155.0,650.0,2304.0,67.0,10.0,NaN
2,University of Alaska Southeast,AK,1,146.0,117.0,89.0,4.0,24.0,492.0,1849.0,1742.0,5226.0,2514.0,2250.0,34.0,500.0,1162.0,39.0,9.5,39.0
3,University of Alaska at Anchorage,AK,1,2065.0,1598.0,1162.0,NaN,NaN,6209.0,10537.0,1742.0,5226.0,2600.0,2520.0,114.0,580.0,1260.0,48.0,13.7,NaN
4,Alabama Agri. & Mech. Univ.,AL,1,2817.0,1920.0,984.0,NaN,NaN,3958.0,305.0,1700.0,3400.0,1108.0,1442.0,155.0,500.0,850.0,53.0,14.3,40.0


In [35]:
list(df.columns)

['College Name',
 'State',
 'Public (1)/ Private (2)',
 "# appli. rec'd",
 '# appl. accepted',
 '# new stud. enrolled',
 '% new stud. from top 10%',
 '% new stud. from top 25%',
 '# FT undergrad',
 '# PT undergrad',
 'in-state tuition',
 'out-of-state tuition',
 'room',
 'board',
 'add. fees',
 'estim. book costs',
 'estim. personal $',
 '% fac. w/PHD',
 'stud./fac. ratio',
 'Graduation rate']

In [36]:
df.columns = [s.strip().replace(' ', '_') for s in df.columns] 
list(df.columns)

['College_Name',
 'State',
 'Public_(1)/_Private_(2)',
 "#_appli._rec'd",
 '#_appl._accepted',
 '#_new_stud._enrolled',
 '%_new_stud._from_top_10%',
 '%_new_stud._from_top_25%',
 '#_FT_undergrad',
 '#_PT_undergrad',
 'in-state_tuition',
 'out-of-state_tuition',
 'room',
 'board',
 'add._fees',
 'estim._book_costs',
 'estim._personal_$',
 '%_fac._w/PHD',
 'stud./fac._ratio',
 'Graduation_rate']

In [37]:
#Finding out number of mising values
df.isnull().sum()


College_Name                  0
State                         0
Public_(1)/_Private_(2)       0
#_appli._rec'd               10
#_appl._accepted             11
#_new_stud._enrolled          5
%_new_stud._from_top_10%    235
%_new_stud._from_top_25%    202
#_FT_undergrad                3
#_PT_undergrad               32
in-state_tuition             30
out-of-state_tuition         20
room                        321
board                       498
add._fees                   274
estim._book_costs            48
estim._personal_$           181
%_fac._w/PHD                 32
stud./fac._ratio              2
Graduation_rate              98
dtype: int64

In [38]:
#PART_A: Excluding categorical variables and eliminating the null values
df1 = df.select_dtypes(exclude=['object']).copy()
df1.head()

,Public_(1)/_Private_(2),#_appli._rec'd,#_appl._accepted,#_new_stud._enrolled,%_new_stud._from_top_10%,%_new_stud._from_top_25%,#_FT_undergrad,#_PT_undergrad,in-state_tuition,out-of-state_tuition,room,board,add._fees,estim._book_costs,estim._personal_$,%_fac._w/PHD,stud./fac._ratio,Graduation_rate
0,2,193.0,146.0,55.0,16.0,44.0,249.0,869.0,7560.0,7560.0,1620.0,2500.0,130.0,800.0,1500.0,76.0,11.9,15.0
1,1,1852.0,1427.0,928.0,NaN,NaN,3885.0,4519.0,1742.0,5226.0,1800.0,1790.0,155.0,650.0,2304.0,67.0,10.0,NaN
2,1,146.0,117.0,89.0,4.0,24.0,492.0,1849.0,1742.0,5226.0,2514.0,2250.0,34.0,500.0,1162.0,39.0,9.5,39.0
3,1,2065.0,1598.0,1162.0,NaN,NaN,6209.0,10537.0,1742.0,5226.0,2600.0,2520.0,114.0,580.0,1260.0,48.0,13.7,NaN
4,1,2817.0,1920.0,984.0,NaN,NaN,3958.0,305.0,1700.0,3400.0,1108.0,1442.0,155.0,500.0,850.0,53.0,14.3,40.0


In [39]:
df1.shape

(1302, 18)

In [40]:
#Now removing null values
df1.dropna()

,Public_(1)/_Private_(2),#_appli._rec'd,#_appl._accepted,#_new_stud._enrolled,%_new_stud._from_top_10%,%_new_stud._from_top_25%,#_FT_undergrad,#_PT_undergrad,in-state_tuition,out-of-state_tuition,room,board,add._fees,estim._book_costs,estim._personal_$,%_fac._w/PHD,stud./fac._ratio,Graduation_rate
0,2,193.0,146.0,55.0,16.0,44.0,249.0,869.0,7560.0,7560.0,1620.0,2500.0,130.0,800.0,1500.0,76.0,11.9,15.0
2,1,146.0,117.0,89.0,4.0,24.0,492.0,1849.0,1742.0,5226.0,2514.0,2250.0,34.0,500.0,1162.0,39.0,9.5,39.0
9,2,805.0,588.0,287.0,67.0,88.0,1376.0,207.0,11660.0,11660.0,2050.0,2430.0,120.0,400.0,900.0,74.0,14.0,72.0
11,2,608.0,520.0,127.0,26.0,47.0,538.0,126.0,8080.0,8080.0,1380.0,2540.0,100.0,500.0,1100.0,63.0,11.4,44.0
21,2,4414.0,1500.0,335.0,30.0,60.0,908.0,119.0,5666.0,5666.0,1424.0,1540.0,418.0,1000.0,1400.0,56.0,15.5,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,1,5729.0,4520.0,1957.0,20.0,52.0,8516.0,1264.0,2208.0,6996.0,1565.0,1140.0,1070.0,279.0,1260.0,80.0,19.3,61.0
1283,2,152.0,128.0,75.0,17.0,41.0,282.0,22.0,9100.0,9100.0,1800.0,1900.0,90.0,500.0,1400.0,48.0,8.5,50.0
1284,2,582.0,498.0,172.0,21.0,44.0,799.0,78.0,10468.0,10468.0,1574.0,1806.0,92.0,660.0,1800.0,40.0,11.5,52.0
1291,2,4226.0,3666.0,2007.0,14.0,60.0,7703.0,2339.0,1660.0,5094.0,2030.0,1980.0,330.0,700.0,1560.0,77.0,16.6,50.0


In [41]:
df1.dropna().shape

(471, 18)

In [43]:
#PART-B- Principal Component Analysis of Normalized & De-Normalized Data
pcs = PCA()
pcs.fit(df1.dropna(axis=0))
pcsSummary_df1 = pd.DataFrame({'Standard deviation': np.sqrt(pcs.explained_variance_),
                           'Proportion of variance': pcs.explained_variance_ratio_,
                           'Cumulative proportion': np.cumsum(pcs.explained_variance_ratio_)})
pcsSummary_df1 = pcsSummary_df1.transpose()
pcsSummary_df1.columns = ['PC{}'.format(i) for i in range(1, len(pcsSummary_df1.columns) + 1)]
pcsSummary_df1.round(4)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
Standard deviation,7430.9140,5987.9890,1854.6412,1192.5293,967.4279,679.6527,596.9761,580.6299,417.6136,318.1272,188.8676,155.6062,19.0491,12.5287,11.0185,5.33,2.9062,0.1692
Proportion of variance,0.5614,0.3645,0.0350,0.0145,0.0095,0.0047,0.0036,0.0034,0.0018,0.0010,0.0004,0.0002,0.0000,0.0000,0.0000,0.00,0.0000,0.0000
Cumulative proportion,0.5614,0.9259,0.9609,0.9753,0.9848,0.9895,0.9932,0.9966,0.9984,0.9994,0.9997,1.0000,1.0000,1.0000,1.0000,1.00,1.0000,1.0000


In [44]:
#De-normalized data
pcsComponents_df1 = pd.DataFrame(pcs.components_.transpose(), columns=pcsSummary_df1.columns, 
                                index=df1.columns)
pcsComponents_df1.iloc[:,:5]


,PC1,PC2,PC3,PC4,PC5
Public_(1)/_Private_(2),-0.000048,7.249543e-07,0.000006,0.000002,-0.000205
#_appli._rec'd,0.271883,5.511834e-01,0.664458,0.129476,-0.034246
#_appl._accepted,0.194107,3.212994e-01,0.190957,-0.008357,-0.076674
#_new_stud._enrolled,0.084730,1.015899e-01,-0.087451,-0.055253,-0.036068
%_new_stud._from_top_10%,-0.000898,1.732235e-03,0.000136,-0.001906,0.001236
%_new_stud._from_top_25%,-0.000811,1.924733e-03,0.000040,-0.002352,0.001009
#_FT_undergrad,0.458121,4.922634e-01,-0.635303,-0.284582,-0.080402
#_PT_undergrad,0.108253,7.340954e-02,-0.285353,0.942562,-0.051743
in-state_tuition,-0.670187,3.824891e-01,-0.082787,-0.016972,-0.621759
out-of-state_tuition,-0.454535,4.286851e-01,-0.129410,0.018657,0.748634


In [45]:
#Normalized data
pcs = PCA()
pcs.fit(preprocessing.scale(df1.dropna(axis=0)))
pcsSummary_df1 = pd.DataFrame({'Standard deviation': np.sqrt(pcs.explained_variance_),
                           'Proportion of variance': pcs.explained_variance_ratio_,
                           'Cumulative proportion': np.cumsum(pcs.explained_variance_ratio_)})
pcsSummary_df1 = pcsSummary_df1.transpose()
pcsSummary_df1.columns = ['PC{}'.format(i) for i in range(1, len(pcsSummary_df1.columns) + 1)]
pcsSummary_df1.round(4)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
Standard deviation,2.3673,2.1907,1.1113,1.0339,0.9919,0.8747,0.8356,0.7736,0.7347,0.6634,0.6305,0.5853,0.4590,0.4382,0.3008,0.1892,0.1473,0.1199
Proportion of variance,0.3107,0.2661,0.0685,0.0593,0.0545,0.0424,0.0387,0.0332,0.0299,0.0244,0.0220,0.0190,0.0117,0.0106,0.0050,0.0020,0.0012,0.0008
Cumulative proportion,0.3107,0.5767,0.6452,0.7045,0.7590,0.8014,0.8401,0.8733,0.9032,0.9276,0.9497,0.9687,0.9804,0.9910,0.9960,0.9980,0.9992,1.0000


In [46]:
pcsComponents_df1 = pd.DataFrame(pcs.components_.transpose(), columns=pcsSummary_df1.columns, 
                                index=df1.columns)
pcsComponents_df1.iloc[:,:5]


,PC1,PC2,PC3,PC4,PC5
Public_(1)/_Private_(2),-0.316591,-0.147475,0.171296,-0.032228,-0.198917
#_appli._rec'd,0.088247,0.405724,0.001790,0.063741,-0.071633
#_appl._accepted,0.139203,0.393314,0.014698,0.104870,-0.152524
#_new_stud._enrolled,0.190780,0.381118,0.005916,-0.042852,-0.122273
%_new_stud._from_top_10%,-0.269376,0.239991,-0.139189,-0.366049,0.180971
%_new_stud._from_top_25%,-0.248772,0.255805,-0.157072,-0.380724,0.170675
#_FT_undergrad,0.209688,0.371112,0.035582,-0.065777,-0.122824
#_PT_undergrad,0.196632,0.205937,0.299609,0.047936,-0.295684
in-state_tuition,-0.396579,0.027634,0.091330,0.036630,-0.164870
out-of-state_tuition,-0.371248,0.131736,0.044006,0.075838,-0.134095
